In [1]:
import os, sys

from math import ceil

import numpy as np
import meshplot as mp
import ipywidgets
from skimage import measure
from scipy.ndimage import zoom
from scipy.interpolate import interpn
from IPython.display import display
from einops import rearrange
import igl
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
import torch
from scipy import stats
import matplotlib.pyplot as plt
import meshio
from sdf import *


In [10]:
#x_coords = np.linspace(-1.5, 1.5, 50)
#y_coords = np.linspace(-1.5, 1.5, 50)
#z_coords = np.linspace(-1, 1, 50)
#x = np.stack(np.meshgrid(x_coords, y_coords, z_coords))

In [11]:
def _length(a):
    return np.linalg.norm(a, axis=0)

In [12]:
_min = np.minimum
_max = np.maximum

def sdf_box(x, height, width, depth):
    q = np.abs(x) - 1
    print(q.shape)
    return _length(_max(q, 0)) + _min(np.amax(q, axis=0), 1)

In [13]:
def center_crop(img, shape):
    start = [a // 2 - da // 2 for a, da in zip(img.shape, shape)]
    end = [a + b for a, b in zip(start, shape)]
    slices = tuple([slice(a, b) for a, b in zip(start, end)])
    return img[slices]

In [14]:
# Add noise to coordinates
def gradient_noise(x, scale, strength, seed=None):
    shape = [ceil(s / scale) for s in x.shape[1:]]
    if seed:
        np.random.seed(seed)
    scalar_noise = np.random.randn(*shape)
    scalar_noise = zoom(scalar_noise, zoom=scale)
    #scalar_noise = center_crop(scalar_noise, shape=x.shape[1:])
    vector_noise = np.stack(np.gradient(scalar_noise))
    return vector_noise * strength

In [15]:
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [16]:
plot=None
@mp.interact(height=(1, 3, 0.5),
             width=(1, 3, 0.5),
             depth=(1, 3, 0.5),
             noise_scale=(1, 1), 
             noise_strength=(0.0, 0.01, 0.001),
             seed=(1, 100)
            )
def show(height, width, depth, noise_scale, noise_strength, seed):
    global plot
    x_coords = np.linspace(-height, height, 50)
    y_coords = np.linspace(-width, width, 50)
    z_coords = np.linspace(-depth, depth, 50)
    x = np.stack(np.meshgrid(x_coords, y_coords, z_coords))
    
    x = x + gradient_noise(x, noise_scale, noise_strength, seed)
    
    #x = x + gradient_noise(x, noise_scale, noise_strength, seed)
    sdf = sdf_box(x, height, width, depth)
    verts, faces, normals, values = measure.marching_cubes(sdf, level=0)
    
    if plot is None:
        plot = mp.plot(verts, faces, return_plot=True)
    else:
        with HiddenPrints():
            plot.update_object(vertices=verts, faces=faces)
        display(plot._renderer)

interactive(children=(FloatSlider(value=2.0, description='height', max=3.0, min=1.0, step=0.5), FloatSlider(va…

In [37]:
plot=None
@mp.interact(height=(1, 3, 0.5),
             width=(1, 3, 0.5),
             depth=(1, 3, 0.5),
             noise_scale=(1, 1), 
             noise_strength=(0.0, 0.01, 0.001),
             seed=(1, 100),
             position=(-0.5, 0.5, 0.1),
             bump_width=(0.01, 0.02, 0.001),
             bump_height=(0.01, 50.)
            )
def show(height, width, depth, noise_scale, noise_strength, seed, position, bump_width, bump_height):
    global plot
    x_coords = np.linspace(-height, height, 50)
    y_coords = np.linspace(-width, width, 50)
    z_coords = np.linspace(-depth, depth, 50)
    x = np.stack(np.meshgrid(x_coords, y_coords, z_coords))
    sdf = sdf_box(x, height, width, depth)
    verts, faces, normals, values = measure.marching_cubes(sdf, level=0)
    
    x_warp = gradient_noise(x, noise_scale, noise_strength, seed)
    
    center = np.array([position, 0., 0.])
    x_dist = np.linalg.norm((x - center[:, None, None, None]), axis=0)
    x_bump = bump_height * np.e ** -(1. / bump_width * x_dist ** 2)
    x_warp += -np.stack(np.gradient(x_bump))
    
    x_warp = rearrange(x_warp, 'v h w d -> h w d v')
    vertex_noise = interpn([np.arange(50) for _ in range(3)], x_warp, verts)
    verts += vertex_noise
    
    #x = x + gradient_noise(x, noise_scale, noise_strength, seed)
    #sdf = sdf_box(x)
    #verts, faces, normals, values = measure.marching_cubes(sdf, level=0)
    
    if plot is None:
        plot = mp.plot(verts, faces, return_plot=True)
    else:
        with HiddenPrints():
            plot.update_object(vertices=verts, faces=faces)
        display(plot._renderer)

interactive(children=(FloatSlider(value=2.0, description='height', max=3.0, min=1.0, step=0.5), FloatSlider(va…

In [7]:
x_coords = np.linspace(-1, 1, 50)
y_coords = np.linspace(-1, 1, 50)
z_coords = np.linspace(-1, 1, 50)
x = np.stack(np.meshgrid(x_coords, y_coords, z_coords))
sdf = sdf_box(x, 1, 1, 1)
verts, faces, normals, values = measure.marching_cubes(sdf, level=0)

(3, 50, 50, 50)


RuntimeError: No surface found at the given iso value.

In [18]:
def get_vertices_faces(points):
    points, cells = np.unique(points, axis=0, return_inverse=True)
    cells = [('triangle', cells.reshape((-1, 3)))]
    mesh = meshio.Mesh(points, cells)
    mesh.write("temp.stl")
    vertices, faces = igl.read_triangle_mesh("temp.stl")
    os.remove("temp.stl")
    return vertices, faces

## Box

In [19]:
a = box((2, 1, 1))
a = a.generate()
a = np.stack(a)
vertices, faces = get_vertices_faces(a)
mp.plot(v=vertices, f=faces)

min -1.05679, -0.621132, -0.621132
max 1.05679, 0.621133, 0.621133
step 0.00919593, 0.00919593, 0.00919593
4645200 samples in 200 batches with 8 workers
  100% (200 of 200) [##############################] 0:00:01 0:00:00    
21 skipped, 87 empty, 92 nonempty
234140 triangles in 0.959407 seconds


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

## Sphere

In [20]:
b = sphere()
b = b.generate()
b = np.stack(b)
vertices, faces = get_vertices_faces(b)
mp.plot(v=vertices, f=faces)

min -1.05679, -1.05679, -1.05679
max 1.05679, 1.05679, 1.05679
step 0.0131058, 0.0131058, 0.0131058
4657463 samples in 216 batches with 8 workers
  100% (216 of 216) [##############################] 0:00:01 0:00:00    
91 skipped, 27 empty, 98 nonempty
219284 triangles in 0.921696 seconds


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

## Half Sphere

In [21]:
f = sphere() & plane()
f = f.generate()
f = np.stack(f)
vertices, faces = get_vertices_faces(f)
mp.plot(v=vertices, f=faces)

min -1.05679, -1.05679, -0.0947075
max 1.05679, 1.05679, 1.13649
step 0.0109455, 0.0109455, 0.0109455
4640000 samples in 196 batches with 8 workers
  100% (196 of 196) [##############################] 0:00:02 0:00:00    
77 skipped, 27 empty, 92 nonempty
209940 triangles in 1.816 seconds


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

## Box with Bump

In [23]:
a = box((2, 1, 1))
b = sphere(radius=0.2, center=(-0.5, 0, 0.5))

#f = a | b.k(0.25)
f = union(a, b, k=0.2) # equivalent

f = f.generate()
f = np.stack(f)
vertices, faces = get_vertices_faces(f)
mp.plot(v=vertices, f=faces)

min -1.05679, -0.621132, -0.641534
max 1.05679, 0.621133, 0.769503
step 0.00959482, 0.00959482, 0.00959482
4623536 samples in 175 batches with 8 workers
  100% (175 of 175) [##############################] 0:00:01 0:00:00    
27 skipped, 52 empty, 96 nonempty
224132 triangles in 1.41249 seconds


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

## Generate & Save Boxes with Bumps

In [4]:
for idx, x_coor in enumerate(tqdm(np.arange(-0.5, 0.5, 0.1))):
    with HiddenPrints():
        a = box((2, 1, 1))
        b = sphere(radius=0.2, center=(x_coor, 0, 0.5))
        f = union(a, b, k=0)

        f = f.generate()
        points = np.stack(f)

        points, cells = np.unique(points, axis=0, return_inverse=True)
        cells = [('triangle', cells.reshape((-1, 3)))]
        mesh = meshio.Mesh(points, cells)
        mesh.write(f"box_bump_100/box_bump_{idx}.stl")
    
    #vertices, faces = get_vertices_faces(f)
    #igl.write_triangle_mesh(f"box_bump_100/box_bump_{idx}.ply", vertices, faces)
    

100%|███████████████████████████████████████████████████████████████████████████████████| 10/10 [00:46<00:00,  4.66s/it]
